In [1]:
# GROUP: Graphkrone
# Members: Marcello Negri     19-945-450
#          Riccardo Uslenghi  19-954-262

import numpy as np 
import pandas as pd
import matplotlib as mpl 
import matplotlib.pyplot as plt 
from tqdm import tqdm

from sklearn import cluster, datasets, mixture
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
import random
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
#from sklearn.experimental import enable_iterative_imputer  
#from sklearn.impute import IterativeImputer

import warnings
warnings.filterwarnings("ignore")

# validation_fractionfloat, default=0.1 FA LA CROSS VAL IN AUTOMATICO

#out_file = open("solutions.txt", "w")

# loading dataset from 'train.csv', skipping first line (labels)
data_trainX = np.loadtxt('train_features.csv', delimiter=",", skiprows=1)
data_trainY = np.loadtxt('train_labels.csv', delimiter=",", skiprows=1)
data_testX = np.loadtxt('test_features.csv', delimiter=",", skiprows=1)

#trainXnn = np.loadtxt('./nn/train_nn_new.csv', delimiter=",", skiprows=1)
#testXnn = np.loadtxt('./nn/test_nn_new.csv', delimiter=",", skiprows=1)


pid = []
for c in range(int(len(data_testX)/12)):
    pid.append(int(data_testX[c*12,0]))

trainY1 = data_trainY[:,1]
trainY2 = data_trainY[:,2]
trainY3 = data_trainY[:,3]
trainY4 = data_trainY[:,4]
trainY5 = data_trainY[:,5]
trainY6 = data_trainY[:,6]
trainY7 = data_trainY[:,7]
trainY8 = data_trainY[:,8]
trainY9 = data_trainY[:,9]
trainY10 = data_trainY[:,10]
trainY11 = data_trainY[:,11]
trainY12 = data_trainY[:,12:]

In [2]:
import copy
from tqdm import tqdm

def preprocessing_zeros(data_train):
    trainX = []
    
    for r in tqdm(range(int(len(data_train)/12))):
        data_matrix = data_train[r*12:(r+1)*12,1:]
        data_array = np.nanmean(data_matrix, axis=0)
        trainX.append(data_array)
     
    trainX = np.array(trainX)
    #trainX = (trainX - np.nanmean(trainX, axis=0))/np.nanstd(trainX, axis=0)
    trainX[np.isnan(trainX)] = 0

    return trainX

def preprocessing(data_train):
    trainX = []
    
    for r in range(int(len(data_train)/12)):
        data_matrix = data_train[r*12:(r+1)*12,1:]
        data_array = np.nanmean(data_matrix, axis=0)
        trainX.append(data_array)
    
    trainX = np.array(trainX)
    #trainX = (trainX - np.nanmean(trainX, axis=0))/np.nanstd(trainX, axis=0)
    #trainX[np.isnan(trainX)] = 0

    return trainX

def delete(trainX, testX):
    indices_columns = []
    
    for c in range(len(trainX[0])):
        unique, count = np.unique(trainX[:,c], return_counts=True)
        counter = dict(zip(unique, count))
        if -999 in counter:
            if counter[-999]/len(trainX[:,c])>0.70:
                indices_columns.append(c)
    
    trainX = np.delete(trainX, indices_columns, axis=1)
    testX = np.delete(testX, indices_columns, axis=1)
    
    return trainX, testX

In [3]:
mlp_range = [4,18,21,24,25,30,34]

trainX = preprocessing(data_trainX) # matrice compressa con nan
attach = np.array(preprocessing_zeros(data_trainX)) # matrice compressa con nan=0

for c in tqdm(mlp_range):
    print(c)
    M = np.c_[attach[:,:c],attach[:,(c+1):]] 
    v = trainX[:,c]
    a = []
    b = []

    for i in range(len(v)):
        if np.isnan(v[i]) == False:
            a.append(M[i])
            b.append(v[i])
    a = np.array(a)
    b = np.array(b)
    
    regressor1 = MLPRegressor(hidden_layer_sizes=(200,100 ), alpha=0.01, max_iter=200, n_iter_no_change=20)
    #regressor1 = ElasticNet()
    regressor1.fit(a, b)
    
    substitute = regressor1.predict(M)
    
    for r in range(len(M)):
        if np.isnan(v[r]):
            trainX[r,c]=substitute[r]
            attach[r,c]=substitute[r]
    for i in range(len(trainX)):
        if np.isnan(trainX[i,c]):
            print("male!!")
    

  0%|          | 0/7 [00:00<?, ?it/s]

4


 14%|█▍        | 1/7 [01:19<07:56, 79.43s/it]

18


 29%|██▊       | 2/7 [02:42<06:43, 80.66s/it]

...

 71%|███████▏  | 5/7 [07:19<02:49, 84.84s/it]

30


 86%|████████▌ | 6/7 [07:53<01:09, 69.38s/it]

34


100%|██████████| 7/7 [09:45<00:00, 83.67s/it]


In [4]:
elastic_range = [7,2,3,5,6,8,9,10,11,12,13,14,15,16,19,20,22,23,26,27,28,29,31,32,33,35]

bad_col = trainX[:,17]
avg = np.nanmean(bad_col)
for i in range(len(bad_col)):
    if np.isnan(bad_col[i]):
        trainX[i,17]=avg

for c in tqdm(elastic_range):
    if c == 35: M = attach[:,:35]
    else: M = np.c_[attach[:,:c],attach[:,(c+1):]] 
    v = trainX[:,c]
    a = []
    b = []

    for i in range(len(v)):
        if np.isnan(v[i]) == False:
            a.append(M[i])
            b.append(v[i])
    a = np.array(a)
    b = np.array(b)
    
    regressor2 = ElasticNet(alpha=0.01, l1_ratio=0.5)
    regressor2.fit(a, b)
    
    substitute = regressor2.predict(M)
    
    for r in range(len(M)):
        if np.isnan(v[r]):
            trainX[r,c]=substitute[r]
            attach[r,c]=substitute[r]
    for i in range(len(trainX)):
        if np.isnan(trainX[i,c]):
            print("male!!")
    

100%|██████████| 26/26 [00:16<00:00,  1.60it/s]


In [5]:
import random
from sklearn.metrics import roc_auc_score

kf = KFold(n_splits=5)
count = 0

for train_index, test_index in kf.split(trainX):
    count += 1
    
    X_train, X_test = trainX[train_index], trainX[test_index]
    Y_train, Y_test = trainY11[train_index], trainY11[test_index]

    X_train = StandardScaler().fit_transform(X_train)
    X_test = StandardScaler().fit_transform(X_test)
    
    classifier11 = SVC(C=1, probability=True, class_weight='balanced', verbose=True)
    classifier11.fit(X_train, Y_train)
    print(roc_auc_score(Y_test, classifier11.predict_proba(X_test)[:,1]))
    if count > 0:
        break
#FINE

[LibSVM]0.6748237072954548


In [13]:
# KFOLD CLASS
from sklearn.ensemble import AdaBoostClassifier

kf = KFold(n_splits=5)
count = 0

for train_index, test_index in kf.split(trainX):
    count += 1
    
    X_train, X_test = trainX[train_index], trainX[test_index]
    Y_train, Y_test = trainY9[train_index], trainY9[test_index]
    
    #X_train, Y_train = presepsis(X_train,Y_train)
    #X_train, Y_train = down_sampling(X_train,Y_train)
    X_train = StandardScaler().fit_transform(X_train)
    X_test = StandardScaler().fit_transform(X_test)
    
    #classifier1 = AdaBoostClassifier()
    #classifier1 = MLPClassifier()
    classifier1 = SVC(C=1, probability=True, class_weight='balanced', verbose=True)
    classifier1.fit(X_train, Y_train)
    print(roc_auc_score(Y_test, classifier1.predict_proba(X_test)[:,1]))
    if count > 0:
        break

[LibSVM]0.6714220395947887


In [ ]:
# KFOLD SVR/ELASTIC-NET
kf = KFold(n_splits=5)

scores = []
count = 0

#train_diff_X = preprocessing_hard(data_trainX)
test_mean_X 

for train_index, test_index in kf.split(trainX):
    count += 1
    X_train, X_test = trainX[train_index], trainX[test_index]
    Y_train, Y_test = trainY15[train_index], trainY15[test_index]
    
    X_train = StandardScaler().fit_transform(X_train)
    X_test = StandardScaler().fit_transform(X_test)    
    
    regressor = ElasticNet(alpha=0.01, l1_ratio=1)
    #regressor = SVR()
    regressor.fit(X_train, Y_train)
    score_vero = 0.5 + 0.5 * np.maximum(0, r2_score(Y_test, regressor.predict(X_test)))
    print('This is your score: ', score_vero)
    if count > 0:
        break

In [6]:
print(trainX[:3,:])

[[ 8.50000000e+00  3.40000000e+01  3.62465604e+01  2.69903865e+01
   1.20000000e+01  1.49985343e+00  3.67500000e+01  8.56666667e+00
   2.53333333e+01 -6.66666667e-01  1.70000000e+01  2.22405418e+02
   4.60000000e+00  5.23333333e+00  5.00000000e-01  4.33333333e+01
   3.59389260e+02  4.25000000e-01  1.43000000e+02  9.47815035e+01
   1.20000000e+02  6.83333333e+01  1.80000000e+00  4.00000000e+00
   5.02500000e+01  7.60000000e+00  4.79251808e+01  1.00000000e+02
  -7.01856554e-01  1.12000000e+02  2.32000000e+01  7.70833333e+01
  -3.26237874e-01 -1.33468921e+01  1.14500000e+02  7.37000000e+00]
 [            nan             nan  3.26523265e+01  4.11021348e+01
              nan  1.70026761e+00  3.64414615e+01  1.39340314e+01
   2.83208924e+01 -2.14537191e+01  1.84408791e+01  3.38438829e+02
   2.99207002e+00  9.89689984e+00  5.87720474e-01  3.70495859e+01
   5.59616388e+01             nan             nan  7.40145149e+01
   1.37589301e+02             nan  1.96643207e+00  3.83525841e+00
         

In [20]:
trainX = preprocessing(data_trainX) # matrice compressa con nan
attach = np.array(preprocessing_zeros(data_trainX)) # matrice compressa con nan=0

for c in tqdm(range(2,36)):
    if c == 35: M = attach[:,:35]
    else: M = np.c_[attach[:,:c],attach[:,(c+1):]] 
    v = trainX[:,c]
    a = []
    b = []

    for i in range(len(v)):
        if np.isnan(v[i]) == False:
            a.append(M[i])
            b.append(v[i])
    a = np.array(a)
    b = np.array(b)
    
    #regressor1 = MLPRegressor(hidden_layer_sizes=(200,100 ), alpha=0.01, max_iter=200, n_iter_no_change=20)
    regressor1 = ElasticNet()
    regressor1.fit(a, b)
    
    substitute = regressor1.predict(M)
    
    for r in range(len(M)):
        if np.isnan(v[r]):
            trainX[r,c]=substitute[r]
            attach[r,c]=substitute[r]
    for i in range(len(trainX)):
        if np.isnan(trainX[i,c]):
            print("male!!")

print(trainX[:10,:])

100%|██████████| 34/34 [00:09<00:00,  3.76it/s]

[[ 8.50000000e+00  3.40000000e+01  3.75964272e+01  3.11757588e+01
   1.20000000e+01  1.34452053e+00  3.67500000e+01  8.56666667e+00
   2.53333333e+01 -6.66666667e-01  1.70000000e+01  2.38452783e+02
   4.60000000e+00  5.23333333e+00  5.00000000e-01  4.33333333e+01
   2.27690511e+02  4.25000000e-01  1.43000000e+02  9.44208118e+01
   1.20000000e+02  6.83333333e+01  1.80000000e+00  4.00000000e+00
   5.02500000e+01  7.60000000e+00  3.98264413e+01  1.00000000e+02
   1.21832159e-01  1.12000000e+02  2.32000000e+01  7.70833333e+01
   8.08218623e-01  2.08232113e+00  1.14500000e+02  7.37000000e+00]
 [ 6.50000000e+00  7.10000000e+01  3.22126482e+01  2.78000000e+01
   1.20000000e+01  1.85722319e+00  3.60000000e+01  1.46000000e+01
   2.92734570e+01  1.10630238e+00  1.80909091e+01  3.16697040e+02
   2.50000000e+00  1.15000000e+01  8.20000000e-01  3.80516962e+01
   2.00000000e+01  2.68215581e+00  2.07000000e+02  8.62377843e+01
   1.52000000e+02  1.01727273e+02  1.50000000e+00  3.20000000e+00
   8.3272

In [112]:
classifier0 = SVC(probability=True, class_weight='balanced')
classifier1 = SVC(probability=True, class_weight='balanced')
classifier2 = SVC(probability=True, class_weight='balanced')
classifier3 = SVC(probability=True, class_weight='balanced')
classifier4 = SVC(probability=True, class_weight='balanced')
classifier5 = SVC(probability=True, class_weight='balanced')
classifier6 = SVC(probability=True, class_weight='balanced')
classifier7 = SVC(probability=True, class_weight='balanced')
classifier8 = SVC(probability=True, class_weight='balanced')
classifier9 = SVC(probability=True, class_weight='balanced')
classifier10 = SVC(probability=True, class_weight='balanced')
classifier11 = SVC(probability=True, class_weight='balanced')

regressor = MLPRegressor(hidden_layer_sizes=(100, ), 
                           activation='logistic',
                           solver='adam',
                           #max_iter=n_iter, 
                           #alpha=np.power(10., reg),
                           verbose=10
                           #tol=1e-4, 
                           #random_state=1,
                           #learning_rate_init=.1
                           )

In [6]:
elastic_range = [2,3,5,6,7,8,9,10,11,12,13,14,15,16,19,20,22,23,26,27,28,29,31,32,33,35]

for c in tqdm(elastic_range):
    if c == 35: M = attach[:,:35]
    else: M = np.c_[attach[:,:c],attach[:,(c+1):]] 
    M = np.c_[attach[:,:c],attach[:,(c+1):]]
    v = trainX[:,c]
    a = []
    b = []

    for i in range(len(v)):
        if np.isnan(v[i]) == False:
            a.append(M[i])
            b.append(v[i])

    kf = KFold(n_splits=10)
    count = 0
    a = np.array(a)
    b = np.array(b)

    for train_index, test_index in kf.split(a):
        count += 1

        X_train, X_test = a[train_index], a[test_index]
        Y_train, Y_test = b[train_index], b[test_index]

        regressor = ElasticNet(alpha=0.01, l1_ratio=0.5)
        regressor.fit(X_train, Y_train)
        
        risultato = r2_score(Y_test, regressor.predict(X_test))
        overfit = r2_score(Y_train, regressor.predict(X_train))

        print('column = {0}, score = {1}, overfit = {2}'.format(c,risultato,overfit))
        if count > 0:
            break

  8%|▊         | 2/25 [00:00<00:03,  7.45it/s]

column = 2, score = 0.14596275152555316, overfit = 0.15650150393005968
column = 3, score = 0.01932831438868754, overfit = 0.0768216017574449


 12%|█▏        | 3/25 [00:00<00:03,  7.18it/s]

column = 5, score = 0.18814097782144723, overfit = 0.2573054599939968
column = 6, score = 0.13796060659077736, overfit = 0.13819374679587937

 16%|█▌        | 4/25 [00:00<00:03,  6.49it/s]

...

column = 29, score = 0.28928917779246144, overfit = 0.2992404449377556


 96%|█████████▌| 24/25 [00:04<00:00,  5.62it/s]

column = 31, score = 0.1808557515947845, overfit = 0.17405764238583932
column = 32, score = 0.1811609567245399, overfit = 0.2861270041607302


100%|██████████| 25/25 [00:04<00:00,  5.74it/s]

column = 33, score = 0.017874820335940234, overfit = 0.07928550300921211


In [4]:
# VA RIMESSO TESTX
classifier0.fit(trainX, trainY1)
M_0 = classifier0.predict_proba(trainX)[:,1]

In [5]:
classifier1.fit(trainX, trainY1)
M_1 = classifier1.predict_proba(testX)[:,1]

In [6]:
classifier2.fit(trainX, trainY2)
M_2 = classifier2.predict_proba(testX)[:,1]

In [7]:
classifier3.fit(trainX, trainY3)
M_3 = classifier3.predict_proba(testX)[:,1]

In [8]:
classifier4.fit(trainX, trainY4)
M_4 = classifier4.predict_proba(testX)[:,1]

In [9]:
classifier5.fit(trainX, trainY5)
M_5 = classifier5.predict_proba(testX)[:,1]

In [10]:
classifier6.fit(trainX, trainY6)
M_6 = classifier6.predict_proba(testX)[:,1]

In [11]:
classifier7.fit(trainX, trainY7)
M_7 = classifier7.predict_proba(testX)[:,1]

In [12]:
classifier8.fit(trainX, trainY8)
M_8 = classifier8.predict_proba(testX)[:,1]

In [13]:
classifier9.fit(trainX, trainY9)
M_9 = classifier9.predict_proba(testX)[:,1]

In [14]:
classifier10.fit(trainX, trainY10)
M_10 = classifier10.predict_proba(testX)[:,1]

In [32]:
# SEPSIS
classifier11.fit(trainX_sepsis, trainY11)
M_11 = classifier11.predict_proba(testX_sepsis)[:,1]

In [33]:
M_train = classifier11.predict_proba(trainX_sepsis)[:,1]

In [17]:
# VITALS
regressor.fit(trainX, trainY12)
M_12 = regressor.predict(testX)

# activation: 'logistic' -> Iteration 200, loss = 11.38286668
# activation: 'relu' -> Iteration 200, loss = 10.59883991
# preprocessing + logistic -> Iteration 200, loss = 18.61017290

Iteration 1, loss = 2815.82433091
Iteration 2, loss = 2467.37367954
Iteration 3, loss = 2111.26558622
Iteration 4, loss = 1760.07395876
Iteration 5, loss = 1428.22039765
Iteration 6, loss = 1127.48040290
Iteration 7, loss = 866.50741490
Iteration 8, loss = 649.61061872
Iteration 9, loss = 476.45358049
Iteration 10, loss = 343.68109495
Iteration 11, loss = 245.66101177
Iteration 12, loss = 176.05152511
Iteration 13, loss = 128.29044072
Iteration 14, loss = 96.61111084
Iteration 15, loss = 76.21423071
Iteration 16, loss = 63.39207169
Iteration 17, loss = 55.49553245
Iteration 18, loss = 50.69117735
Iteration 19, loss = 47.74682866
Iteration 20, loss = 45.92066468
Iteration 21, loss = 44.74037671
Iteration 22, loss = 43.93422123
Iteration 23, loss = 43.35595614
Iteration 24, loss = 42.90653528
Iteration 25, loss = 42.54272207
Iteration 26, loss = 42.24286667
Iteration 27, loss = 41.97098815
Iteration 28, loss = 41.72355924
Iteration 29, loss = 41.49837414
Iteration 30, loss = 41.27708349


In [18]:
VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']

#Mzeros = np.zeros((12664,4))

M = np.c_[ pid, M_1]
M = np.c_[ M, M_2] 
M = np.c_[ M, M_3]
M = np.c_[ M, M_4] 
M = np.c_[ M, M_5]
M = np.c_[ M, M_6] 
M = np.c_[ M, M_7]
M = np.c_[ M, M_8] 
M = np.c_[ M, M_9]
M = np.c_[ M, M_10] 
M = np.c_[ M, M_11]
M = np.c_[ M, M_12]

'''
pid_train = []

for c in range(int(len(data_trainX)/12)):
    pid_train.append(int(data_trainX[c*12,0]))
'''

df_predictions = pd.DataFrame(M, columns=['pid']+TESTS+['LABEL_Sepsis']+VITALS)
df_predictions.to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')

# Printo la previsione sul train
#pd.DataFrame(Mt, columns=['pid']+TESTS+['LABEL_Sepsis']+VITALS)
#pd.DataFrame(M, columns=['pid']+TESTS+['LABEL_Sepsis']+VITALS)

In [36]:
MM = np.c_[M_train]

In [37]:
pd.DataFrame(M_train[:50])

,0
0,0.076771
1,0.039320
2,0.062809
3,0.090523
4,0.032445
5,0.168708
6,0.028812
7,0.029314
8,0.089283
9,0.096565


In [ ]:
import pandas as pd
import sklearn.metrics as metrics


def get_score(df_true, df_submission):
    df_submission = df_submission.sort_values('pid')
    df_true = df_true.sort_values('pid')
    #task1 = np.mean([metrics.roc_auc_score(df_true[entry], df_submission[entry]) for entry in TESTS])
    #task2 = metrics.roc_auc_score(df_true['LABEL_Sepsis'], df_submission['LABEL_Sepsis'])
    task3 = np.mean([0.5 + 0.5 * np.maximum(0, metrics.r2_score(df_true[entry], df_submission[entry])) for entry in VITALS])
    #score = np.mean([task1, task2, task3])
    score = task3
    #print(task1, task2, task3)
    print(task3)
    return score


filename = 'sample.zip'
df_submission = pd.read_csv(filename)

# generate a baseline based on sample.zip
df_true = pd.read_csv(filename)
for label in TESTS + ['LABEL_Sepsis']:
    # round classification labels
    df_true[label] = np.around(df_true[label].values)

print('Score of sample.zip with itself as groundtruth', get_score(df_true, df_submission))


In [ ]:

def mlp(dataset, hidden_layer_sizes, activation, solver, reg, noise):
    np.random.seed(420)
    classifier = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, 
                               activation=activation,
                               solver=solver,
#                                max_iter=n_iter, 
                               alpha=np.power(10., reg),
#                                verbose=10, 
#                                tol=1e-4, 
                               random_state=1,
                               learning_rate_init=.1)

"""hidden_layer_size=[(1,), (5, ), (50, ), (100, ), (1000, ),
                                   (5, 5, ), (50, 50, ), (100, 100), 
                                   (50, 50, 50), (100, 100, 100)],"""


X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.4)
    
classifier.fit(X_train, y_train)s
print(classifier.score(X_test, y_test))

In [ ]:
def preprocessing_2rows (data_train):
    #overall_mean = np.nanmean(data_train[:,1:], axis=0)
    trainX = []
    
    for r in range(int(len(data_train)/12)):
        a = []
        data_matrix_1 = data_train[r*12:int((r+0.5)*12),1:]
        data_matrix_2 = data_train[int((r+0.5)*12):(r+1)*12,1:]
        data_array_1 = np.nanmean(data_matrix_1, axis=0)
        data_array_2 = np.nanmean(data_matrix_2, axis=0)
        data_array_1[np.isnan(data_array_1)] = 0 # overall_mean[np.isnan(data_array)]
        data_array_2[np.isnan(data_array_2)] = 0
        a.append(data_array_1)
        a.append(data_array_2)
        trainX.append(np.ravel(a))
    
    return trainX

def preprocessing (data_train):
    #overall_mean = np.nanmean(data_train[:,1:], axis=0)
    trainX = []
    
    for r in range(int(len(data_train)/12)):
        data_matrix = data_train[r*12:(r+1)*12,1:]
        data_array = np.nanmean(data_matrix, axis=0)
        data_array[np.isnan(data_array)] = -999 # overall_mean[np.isnan(data_array)]
        trainX.append(data_array)
    
    return trainX

def preprocessing_easy (data_train):
    #overall_mean = np.nanmean(data_train[:,1:], axis=0)
    trainX = []
    indices_columns = []
    indices_rows = []
    
    for r in range(int(len(data_train)/12)):
        data_matrix = data_train[r*12:(r+1)*12,1:]
        data_array = np.nanmean(data_matrix, axis=0)
        #data_array[np.isnan(data_array)] = -999 # overall_mean[np.isnan(data_array)]
        trainX.append(data_array)
        
    trainX = np.array(trainX)
    train_copy = np.array(trainX)
    train_copy[np.isnan(train_copy)] = -999
    
    for c in range(len(trainX[0])):
        unique, count = np.unique(train_copy[:,c], return_counts=True)
        counter = dict(zip(unique, count))
        if -999 in counter:
            if counter[-999]/len(train_copy[:,c])>0.5:
                indices_columns.append(c)
    trainX = np.delete(trainX, indices_columns, axis=1)
    
    #print(pd.DataFrame(trainX))
    
    overall_mean = np.nanmean(trainX, axis=0)
    train = []
    for r in range(len(trainX)):
        data_row = trainX[r,:]
        data_row[np.isnan(data_row)] = 0 #overall_mean[np.isnan(data_row)]
        train.append(data_row)
    
    #print(pd.DataFrame(train))
    
    return train


In [ ]:
def preprocessing(data_train):
        #overall_mean = np.nanmean(data_train[:,1:], axis=0)
    trainX = []
    
    for r in range(int(len(data_train)/12)):
        data_matrix = data_train[r*12:(r+1)*12,1:]
        data_array = np.nanmean(data_matrix, axis=0)
        trainX.append(data_array)
 
    trainX = (trainX - np.nanmean(trainX, axis=0))/np.nanstd(trainX, axis=0)
    trainX[np.isnan(trainX)] = -999
    
    return trainX

def delete(trainX, testX):
    indices_columns = []
    
    for c in range(len(trainX[0])):
        unique, count = np.unique(trainX[:,c], return_counts=True)
        counter = dict(zip(unique, count))
        if -999 in counter:
            if counter[-999]/len(trainX[:,c])>0.70:
                indices_columns.append(c)
    
    trainX = np.delete(trainX, indices_columns, axis=1)
    testX = np.delete(testX, indices_columns, axis=1)
    
    return trainX, testX
    
    
def preprocessing_nn (trainX):
    
    train_copy = trainX[:]
    
    def distance (matrix, r, s, alpha):
        sum_, n_ = 0, 0
        
        for i in range(len(matrix[0])):
            if matrix[r,i] != -999 and matrix[s,i] != -999:
                sum_ += np.square(matrix[r,i]-matrix[s,i])
                
                n_ += 1
        
        if n_ >= 4:
            return (n_**-alpha)*sum_
        else:
            return False
        
    for c in tqdm(range(len(trainX[0]))):#len(trainX[0]):
        for r in range(len(trainX)):
            #if r%1000 == 0: print(r)
            if trainX[r,c] == -999:
                minimum = 1000
                index_min = -1
                count = 0
                count_check = 0
                s = r
                while (count<100 and count_check<500):
                    s += 1
                    count_check += 1
                    s = s%len(trainX)
                    if trainX[s,c] != -999:
                        dist = distance(trainX, r, s, 2)
                        if dist:
                            count += 1
                            if dist < minimum:
                                minimum = dist
                                index_min = s
                if count_check==500:
                    print('--------------------------> ',count_check)
                    print('This is r: ', r)
                    print(trainX[r,:])
                    train_copy[r,:] = np.zeros(len(trainX[0]))
                else:
                    train_copy[r,c] = trainX[index_min,c] #kick random??
                    if trainX[index_min,c]==-999: print("WARNING: substituing -999! ", train_copy[r,c])

    return train_copy

def presepsis (trainX, trainY2):
    trainX_sep = []
    trainY2_sep = []
    #unbalance = numpy.count_nonzero(a, axis=None)[source]/len(prep_trainX)
    
    for r in range(int(len(trainX))):
        if trainY2[r] == 0:
            trainX_sep.append(trainX[r])
            trainY2_sep.append(trainY2[r])
        else:
            for s in range(10):
                trainX_sep.append(trainX[r])
                trainY2_sep.append(trainY2[r])
    
    return trainX_sep, trainY2_sep


  0%|          | 0/26 [00:00<?, ?it/s]

[ 4.62338574e-01 -1.70642596e+00 -9.99000000e+02 -5.55684775e-01
 -3.58821385e-02 -1.05920327e+00  3.63105663e-01 -2.81148876e-01
  7.49499010e-01 -6.94165878e-01 -5.11979019e-01  2.78323900e-01
 -2.15850856e-02 -6.56061723e-01 -3.90102865e-01 -1.03452538e+00
 -5.00153288e-01 -1.88862683e-01 -1.20593914e+00 -6.86480218e-02
  1.07301695e+00  1.06502721e+00 -1.51879305e+00 -4.49558578e-01
 -4.16636694e-01 -8.20539698e-02]
-------------------------->  500
This is r:  13384
[-1.60087537e-01  1.17080747e-01 -9.99000000e+02 -9.99000000e+02
 -9.99000000e+02 -9.99000000e+02 -9.99000000e+02 -9.99000000e+02
 -9.99000000e+02 -9.99000000e+02 -9.99000000e+02 -9.99000000e+02
 -9.99000000e+02 -9.99000000e+02 -9.99000000e+02 -9.99000000e+02
 -9.99000000e+02 -9.99000000e+02 -9.99000000e+02 -9.99000000e+02
 -9.99000000e+02 -9.99000000e+02 -9.99000000e+02 -9.99000000e+02
 -9.99000000e+02 -9.99000000e+02]
-------------------------->  500
This is r:  17101
[-1.60087537e-01 -1.22015751e+00 -9.99000000e+02 -

 23%|██▎       | 6/26 [02:03<06:48, 20.45s/it]

In [128]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit([[np.nan, 2, 3], [4, np.nan, 6], [10, 5, 9]])
imp_mean.transform(X)

array([[-1.99860896,  2.        ,  3.        ],
       [ 4.        ,  3.49913153,  6.        ],
       [10.        ,  4.99999933,  9.        ]])

In [8]:
a = [[1,2],[3,4]]
np.ravel(a)

array([1, 2, 3, 4])